### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

/Users/samriddhimangal/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"


In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    try:
        # create query with a page number
        # API results show 10 articles at a time
        # Make a "GET" request and retrieve the JSON
        # Try and save the reviews to the reviews_list
        response_pages=requests.get(f"{query_url}&page={page}") 
        data=response_pages.json()
        if 'response' in data and 'docs' in data['response']:
            docs=data['response']['docs']
            if docs:
                for doc in docs:
                    # loop through the reviews["response"]["docs"] and append each review to the list
                    reviews_list.append(doc)
                    # Print the page that was just retrieved
                print(f"Checked page {page}")
            else:
                 # Print the page number that had no results then break from the loop
                print(f"Page {page} had no results. Exiting the loop.")
                break
        else:
            print(f"Unexpected response structure on page {page}.")
            break
    except requests.exceptions.RequestException as e:
        print(f"Request failed on page {page}: {e}")
        break
    # Add a twelve second interval between queries to stay within API query limits
    
    time.sleep(12)



Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df=pd.json_normalize(reviews_list)
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# Function to extract the title from the "headline.main" field

def extract_title(headline):
    # Define the pattern to match both kinds of titles
    pattern = r'\u2018(.*?)\u2019 Review|Review: (.*?)$'
    
    # Search for the pattern in the headline
    title_match = re.search(pattern, headline)
    
    if title_match:
        # Check which group matched and return the appropriate group with " Review"
        if title_match.group(1):
            return title_match.group(1)
        elif title_match.group(2):
            return title_match.group(2)
    
    return 'Title not found'

# Apply the function to the "headline.main" column to create a new "title" column
reviews_list_df['title'] = reviews_list_df['headline.main'].apply(extract_title)

# Print the DataFrame
reviews_list_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,‘What’s Love Got to Do With It?’ Probably a Lot
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,A Combustible Pair Find Love in ‘The Other Half’
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,"Love as the World Wars, in ‘The Ottoman Lieute..."
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,It’s All Mirth and Taxes in ‘Love & Taxes’
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebod

In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df['keywords'] = reviews_list_df['keywords'].apply(extract_keywords)

# Print the DataFrame
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,‘What’s Love Got to Do With It?’ Probably a Lot
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,A Combustible Pair Find Love in ‘The Other Half’
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,"Love as the World Wars, in ‘The Ottoman Lieute..."
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,It’s All Mirth and Taxes in ‘Love & Taxes’
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,No

In [10]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list=reviews_list_df['title'].to_list()
title_list

['The Attachment Diaries',
 '‘What’s Love Got to Do With It?’ Probably a Lot',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink C

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url_tmdb = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [12]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)  # Sleep for 10 seconds to avoid hitting the API rate limit
    
    # Add 1 to the request counter
    request_counter += 1
    title1=title.replace(" ", "+")
    # Perform a "GET" request for The Movie Database
    query_url = url_tmdb + title1 + tmdb_key_string
    
    try:
        response = requests.get(query_url).json()
        
        # Include a try clause to search for the full movie details.
        # Use the except clause to print out a statement if a movie is not found.
        if response['results']:
            # Get movie id
            movie_id = response['results'][0]['id']
            
            # Make a request for the full movie details
            movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            
            # Execute "GET" request with url
            movie_details_response = requests.get(movie_details_url).json()
            
            # Extract the genre names into a list
            genres = [genre['name'] for genre in movie_details_response['genres']]
            
            # Extract the spoken_languages' English name into a list
            spoken_languages = [language['english_name'] for language in movie_details_response['spoken_languages']]
            
            # Extract the production_countries' name into a list
            production_countries = [country['name'] for country in movie_details_response['production_countries']]
            
            # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
            movie_data = {
                'title': movie_details_response['title'],
                'original_title':movie_details_response['original_title'],
                'budget':movie_details_response['budget'],
                'homepage':movie_details_response['homepage'],
                'overview':movie_details_response['overview'],
                'popularity':movie_details_response['popularity'],
                'runtime':movie_details_response['runtime'],
                'runtime':movie_details_response['runtime'],
                'revenue': movie_details_response['revenue'],
                'release_date': movie_details_response['release_date'],
                'vote_average': movie_details_response['vote_average'],
                'vote_count': movie_details_response['vote_count'],
                'genres': genres,
                'spoken_languages': spoken_languages,
                'original_language': movie_details_response['original_language'],
                'production_countries': production_countries
            }
            tmdb_movies_list.append(movie_data)
            
            # Print out the title that was found
            print(f"Found movie: {movie_details_response['title']}")
        else:
            print(f"No results found for {title}")
    except Exception as e:
        print(f"Error retrieving data for {title}: {e}")



Found movie: The Attachment Diaries
No results found for ‘What’s Love Got to Do With It?’ Probably a Lot
Found movie: You Can Live Forever
Found movie: A Tourist's Guide to Love
Found movie: Other People's Children
Found movie: One True Loves
Found movie: The Lost Weekend: A Love Story
Found movie: A Thousand and One
Found movie: Your Place or Mine
Found movie: Love in the Time of Fentanyl
Found movie: Pamela, A Love Story
Found movie: In from the Side
Found movie: After Love
Found movie: Alcarràs
Found movie: Nelly and Monsieur Arnaud
Found movie: Lady Chatterley's Lover
Found movie: The Sound of Christmas
Found movie: The Inspection
Found movie: Bones and All
Found movie: My Policeman
Found movie: About Fate
Found movie: Waiting for Bojangles
Found movie: I Love My Dad
Found movie: A Love Song
Found movie: Alone Together
Found movie: Art of Love
Found movie: 2099: The Soldier Protocol
Found movie: Thor: Love and Thunder
Found movie: Both Sides of the Blade
Found movie: Fire of Love
F

No results found for ‘Frantz,’ a Mysterious Frenchman and the Wounds of War
No results found for A Combustible Pair Find Love in ‘The Other Half’
No results found for Love as the World Wars, in ‘The Ottoman Lieutenant’
No results found for It’s All Mirth and Taxes in ‘Love & Taxes’
No results found for ‘Everybody Loves Somebody,’ a Rom-Com With Bite
No results found for Cute Cats of ‘Kedi,’ Rekindling a ‘Love of Life’


In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.789,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "original_language": "es",
        "production_countries": [
            "Argentina"
        ]
 

In [14]:
# Convert the results to a DataFrame
movie_list_df=pd.json_normalize(tmdb_movies_list)
movie_list_df

,title,original_title,budget,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,original_language,production_countries
0,The Attachment Diaries,El apego,0,,"Argentina, 1970s. A desperate young woman goes...",1.789,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],es,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",27.949,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]",en,"[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",13.948,96,0,2023-04-21,6.289,166,"[Romance, Comedy]","[English, Vietnamese]",en,[United States of America]
3,Other People's Children,Les Enfants des autres,0,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",26.688,104,84178,2022-09-21,6.814,191,"[Drama, Comedy]","[French, English]",fr,[France]
4,One True Loves,One True Loves,0,,Emma and Jesse are living the perfect life tog...,16.843,100,37820,2023-04-07,6.513,77,"[Romance, Comedy, Drama]","[English, Spanish]",en,"[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Sicilian Ghost Story,Sicilian Ghost Story,0,,In a little Sicilian village at the edge of a ...,6.159,122,0,2017-11-18,6.600,137,"[Mystery, Drama, Fantasy]",[Italian],it,"[France, Italy, Switzerland]"
137,The New Romantic,The New Romantic,0,,In order to escape her looming post-graduation...,9.514,82,0,2018-10-19,6.300,108,"[Drama, Comedy]",[English],en,[Canada]
138,Pimp,Pimp,0,,An urban love story set on the hard streets of...,8.788,86,0,2018-11-09,6.300,29,[Drama],[English],en,[United States of America]
139,In a Relationship,In a Relationship,0,,Long-term couple Owen and Hallie are breaking ...,9.864,92,0,2018-11-09,5.800,103,"[Drama, Comedy, Romance]",[English],en,"[Mexico, United States of America]"


### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title

merged_ny_tmdb=pd.merge(reviews_list_df, movie_list_df, on='title')
merged_ny_tmdb

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,original_language,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,1.789,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],es,[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,27.949,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]",en,"[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,16.843,100,37820,2023-04-07,6.513,77,"[Romance, Comedy, Drama]","[English, Spanish]",en,"[Czech Republic, United States of America]"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,33.748,95,77145,2023-04-13,6.000,2,[Documentary],[English],en,[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,13.474,116,3395595,2023-03-31,6.871,89,"[Drama, Crime]","[English, Spanish, Portuguese]",en,[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,https://www.nytimes.com/2018/12/06/movies/ashe...,Ron Perlman and Famke Janssen star in this fil...,The New York Times,subject: Movies;creative_works: Asher (Movie);...,2018-12-06T12:00:04+0000,259,‘Asher’ Review: An Aging Hit Man Finds Love,None,None,Asher,...,11.493,117,0,2018-12-07,5.809,138,"[Thriller, Action, Drama]",[English],en,[United States of America]
110,https://www.nytimes.com/2018/11/29/movies/sici...,Flipping from bucolic to brutal with distressi...,The New York Times,subject: Movies;creative_works: Sicilian Ghost...,2018-11-29T12:00:06+0000,258,‘Sicilian Ghost Story’ Review: Fairy-Tale Love...,Critic’s Pick,None,Love in an Enchanted Forest Turns Eerie,...,6.159,122,0,2017-11-18,6.600,137,"[Mystery, Drama, Fantasy]",[Italian],it,"[France, Italy, Switzerland]"
111,https://www.nytimes.com/2018/11/08/movies/the-...,A college newspaper columnist writes about her...,The New York Times,subject: Movies;creative_works: The New Romant...,2018-11-08T12:05:01+0000,269,‘The New Romantic’ Review: Experiments With Lo...,None,None,The New Romantic,...,9.514,82,0,2018-10-19,6.300,108,"[Drama, Comedy]",[English],en,[Canada]
112,https://www.nytimes.com/2018/11/08/movies/pimp...,This lurid melodrama follows a female pimp (Ke...,The New York Times,"subject: Movies;persons: Palmer, Keke;persons:...",2018-11-08T12:00:01+0000,270,‘Pimp’ Review: Blaxploitation and Lesbian Love,None,None,Pimp,...,8.788,86,0,2018-11-09,6.300,29,[Drama],[English],en,[United States of America]


In [16]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['byline.person', 'genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", ":", "}", "{", "'",]

# Loop through the list of columns to fix
for col in columns_to_fix:
    merged_ny_tmdb[col]=merged_ny_tmdb[col].astype(str) # Convert the column to type 'str'
    for char in characters_to_remove:   # Loop through characters to remove
        merged_ny_tmdb[col]=merged_ny_tmdb[col].str.replace(char, "")




# Display the fixed DataFrame
merged_ny_tmdb.head(10)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,original_language,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,1.789,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,es,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,27.949,96,15055,2023-03-24,6.474,38,"Drama, Romance","English, French",en,"Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,16.843,100,37820,2023-04-07,6.513,77,"Romance, Comedy, Drama","English, Spanish",en,"Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,33.748,95,77145,2023-04-13,6.000,2,Documentary,English,en,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,13.474,116,3395595,2023-03-31,6.871,89,"Drama, Crime","English, Spanish, Portuguese",en,United States of America
5,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,...,20.817,109,0,2023-02-10,6.281,724,"Romance, Comedy",English,en,United States of America
6,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,...,1.763,85,0,2023-02-03,0.000,0,Documentary,English,en,"Canada, United States of America"
7,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,subject: Movies;creative_works: After Love (20...,2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,...,12.590,89,0,2021-06-04,7.165,124,Drama,"English, Arabic, French, Urdu",en,United Kingdom
8,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,subject: Movies;creative_works: After Love (20...,2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,...,16.826,98,0,2016-06-08,6.100,115,Drama,French,fr,"Belgium, France"
9,https://www.nytimes.com/2023/01/05/movies/alca...,"In this naturalistic drama from Spain, a famil...",The New York Times,"subject: Movies;persons: Simon, Carla;creative...",2023-01-05T12:00:03+0000,306,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,...,16.984,115,2,2022-04-29,6.900,182,Drama,Catalan,

In [17]:
merged_ny_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  114 non-null    object 
 1   snippet                  114 non-null    object 
 2   source                   114 non-null    object 
 3   keywords                 114 non-null    object 
 4   pub_date                 114 non-null    object 
 5   word_count               114 non-null    int64  
 6   headline.main            114 non-null    object 
 7   headline.kicker          36 non-null     object 
 8   headline.content_kicker  0 non-null      object 
 9   headline.print_headline  113 non-null    object 
 10  headline.name            0 non-null      object 
 11  headline.seo             0 non-null      object 
 12  headline.sub             0 non-null      object 
 13  byline.original          114 non-null    object 
 14  byline.person            1

In [18]:
# Drop "byline.person" column
merged_ny_tmdb.drop(['byline.person'],axis=1, inplace=True)


In [19]:
merged_ny_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  114 non-null    object 
 1   snippet                  114 non-null    object 
 2   source                   114 non-null    object 
 3   keywords                 114 non-null    object 
 4   pub_date                 114 non-null    object 
 5   word_count               114 non-null    int64  
 6   headline.main            114 non-null    object 
 7   headline.kicker          36 non-null     object 
 8   headline.content_kicker  0 non-null      object 
 9   headline.print_headline  113 non-null    object 
 10  headline.name            0 non-null      object 
 11  headline.seo             0 non-null      object 
 12  headline.sub             0 non-null      object 
 13  byline.original          114 non-null    object 
 14  byline.organization      0

In [20]:
# Drop all null values
merged_ny_tmdb_clean = merged_ny_tmdb.dropna(how='all', axis=1)
merged_ny_tmdb_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  114 non-null    object 
 1   snippet                  114 non-null    object 
 2   source                   114 non-null    object 
 3   keywords                 114 non-null    object 
 4   pub_date                 114 non-null    object 
 5   word_count               114 non-null    int64  
 6   headline.main            114 non-null    object 
 7   headline.kicker          36 non-null     object 
 8   headline.print_headline  113 non-null    object 
 9   byline.original          114 non-null    object 
 10  title                    114 non-null    object 
 11  original_title           114 non-null    object 
 12  budget                   114 non-null    int64  
 13  homepage                 114 non-null    object 
 14  overview                 1

In [21]:
# Drop 'headline.kicker' as it mostly has null values
merged_ny_tmdb_clean.drop(['headline.kicker'],axis=1, inplace=True)

/var/folders/vv/72370rd92t5b__qqdr_cdls80000gn/T/ipykernel_2955/2924564226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_ny_tmdb_clean.drop(['headline.kicker'],axis=1, inplace=True)


In [22]:
merged_ny_tmdb_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  114 non-null    object 
 1   snippet                  114 non-null    object 
 2   source                   114 non-null    object 
 3   keywords                 114 non-null    object 
 4   pub_date                 114 non-null    object 
 5   word_count               114 non-null    int64  
 6   headline.main            114 non-null    object 
 7   headline.print_headline  113 non-null    object 
 8   byline.original          114 non-null    object 
 9   title                    114 non-null    object 
 10  original_title           114 non-null    object 
 11  budget                   114 non-null    int64  
 12  homepage                 114 non-null    object 
 13  overview                 114 non-null    object 
 14  popularity               1

In [23]:
# Delete duplicate rows
merged_ny_tmdb_clean = merged_ny_tmdb_clean.drop_duplicates()

In [24]:
merged_ny_tmdb_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113 entries, 0 to 113
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  113 non-null    object 
 1   snippet                  113 non-null    object 
 2   source                   113 non-null    object 
 3   keywords                 113 non-null    object 
 4   pub_date                 113 non-null    object 
 5   word_count               113 non-null    int64  
 6   headline.main            113 non-null    object 
 7   headline.print_headline  112 non-null    object 
 8   byline.original          113 non-null    object 
 9   title                    113 non-null    object 
 10  original_title           113 non-null    object 
 11  budget                   113 non-null    int64  
 12  homepage                 113 non-null    object 
 13  overview                 113 non-null    object 
 14  popularity               113 no

In [25]:
#reset index
merged_ny_tmdb_clean_reset = merged_ny_tmdb_clean.reset_index(drop=True)
merged_ny_tmdb_clean_reset

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.print_headline,byline.original,title,...,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,original_language,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",The Attachment Diaries,By Jeannette Catsoulis,The Attachment Diaries,...,1.789,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,es,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Live Forever,By Elisabeth Vincentelli,You Can Live Forever,...,27.949,96,15055,2023-03-24,6.474,38,"Drama, Romance","English, French",en,"Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,One True Loves,By Brandon Yu,One True Loves,...,16.843,100,37820,2023-04-07,6.513,77,"Romance, Comedy, Drama","English, Spanish",en,"Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,The Lost Weekend: A Love Story,By Glenn Kenny,The Lost Weekend: A Love Story,...,33.748,95,77145,2023-04-13,6.000,2,Documentary,English,en,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,An Unbending Will Meets a Shifting City,By Manohla Dargis,A Thousand and One,...,13.474,116,3395595,2023-03-31,6.871,89,"Drama, Crime","English, Spanish, Portuguese",en,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,https://www.nytimes.com/2018/12/06/movies/ashe...,Ron Perlman and Famke Janssen star in this fil...,The New York Times,subject: Movies;creative_works: Asher (Movie);...,2018-12-06T12:00:04+0000,259,‘Asher’ Review: An Aging Hit Man Finds Love,Asher,By Glenn Kenny,Asher,...,11.493,117,0,2018-12-07,5.809,138,"Thriller, Action, Drama",English,en,United States of America
109,https://www.nytimes.com/2018/11/29/movies/sici...,Flipping from bucolic to brutal with distressi...,The New York Times,subject: Movies;creative_works: Sicilian Ghost...,2018-11-29T12:00:06+0000,258,‘Sicilian Ghost Story’ Review: Fairy-Tale Love...,Love in an Enchanted Forest Turns Eerie,By Jeannette Catsoulis,Sicilian Ghost Story,...,6.159,122,0,2017-11-18,6.600,137,"Mystery, Drama, Fantasy",Italian,it,"France, Italy, Switzerland"
110,https://www.nytimes.com/2018/11/08/movies/the-...,A college newspaper columnist writes about her...,The New York Times,subject: Movies;creative_works: The New Romant...,2018-11-08T12:05:01+0000,269,‘The New Romantic’ Review: Experiments With Lo...,The New Romantic,By Teo Bugbee,The New Romantic,...,9.514,82,0,2018-10-19,6.300,108,"Drama, Comedy",English,en,Canada
111,https://www.nytimes.com/2018/11/08/movies/pimp...,This lurid melodrama follows a female pimp (Ke...,The New York Times,"subject: Movies;persons: Palmer, Keke;persons:...",2018-11-08T12:00:01+0000,270,‘Pimp’ Review: Blaxploitation and Lesbian Love,Pimp,By Jeannette Catsoulis,Pimp,...,8.788,86,0,2018-11-09,6.300,29,Drama,English,en,United States of America


In [26]:
merged_ny_tmdb_clean_reset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  113 non-null    object 
 1   snippet                  113 non-null    object 
 2   source                   113 non-null    object 
 3   keywords                 113 non-null    object 
 4   pub_date                 113 non-null    object 
 5   word_count               113 non-null    int64  
 6   headline.main            113 non-null    object 
 7   headline.print_headline  112 non-null    object 
 8   byline.original          113 non-null    object 
 9   title                    113 non-null    object 
 10  original_title           113 non-null    object 
 11  budget                   113 non-null    int64  
 12  homepage                 113 non-null    object 
 13  overview                 113 non-null    object 
 14  popularity               1

In [27]:
# Export data to CSV without the index
merged_ny_tmdb_clean_reset.to_csv("output/Collected_data.csv", index=False)